In [19]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize

import os
import random

In [2]:
df = pd.read_csv('All-seasons.csv')

In [3]:
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


Выбираем наиболее "разговорчивых персонажей".

In [4]:
Cartman_P= df[df['Character'] == 'Cartman']
Stan_P = df[df['Character'] == 'Stan']
Kyle_P = df[df['Character'] == 'Kyle']
Butters_P = df[df['Character'] == 'Butters']
Randy_P = df[df['Character'] == 'Randy']
print(Cartman_P.describe(), Stan_P.describe(), Kyle_P.describe(), Butters_P.describe(),Randy_P.describe())

       Season Episode Character     Line
count    9774    9774      9774     9774
unique     18      18         1     9340
top         4       7   Cartman  What?\n
freq      801     850      9774       52        Season Episode Character     Line
count    7680    7680      7680     7680
unique     18      18         1     6995
top         2      10      Stan  What?\n
freq      831     594      7680       73        Season Episode Character     Line
count    7099    7099      7099     7099
unique     18      18         1     6493
top         2       1      Kyle  What?\n
freq      824     635      7099       61        Season Episode Character    Line
count    2602    2602      2602    2602
unique     17      17         1    2499
top         6      14   Butters  Huh?\n
freq      378     281      2602      16        Season Episode Character     Line
count    2467    2467      2467     2467
unique     18      17         1     2406
top        14       8     Randy  What?\n
freq      224     259

Уравниваем, достиагем баланса в репликах для более достоверногот результата.

In [6]:
Cartman_P = Cartman_P[:2400]
Stan_P = Stan_P[:2400]
Kyle_P = Kyle_P[:2400]
Butters_P = Butters_P[:2400]
Randy_P = Randy_P[:2400]
talkativecharacters = pd.concat([Cartman_P, Stan_P, Kyle_P, Butters_P,Randy_P])


In [7]:
talkativecharacters.groupby('Character').describe()

Episode     Line Season
Character                               
Butters   count     2400     2400   2400
          unique      17     2307     16
          top         14   Huh?\n      6
          freq       279       13    378
Cartman   count     2400     2400   2400
          unique      14     2359      6
          top          8  What?\n     10
          freq       317       12    574
Kyle      count     2400     2400   2400
          unique      14     2259      9
          top          1  What?\n     10
          freq       242       31    355
Randy     count     2400     2400   2400
          unique      17     2341     18
          top          8  What?\n     14
          freq       259       12    224
Stan      count     2400     2400   2400
          unique      14     2269     10
          top          2  What?\n     16
          freq       230       28    326

Начинаем обучение!

In [13]:
X_train, X_test, y_train, y_test = train_test_split(talkativecharacters['Line'], talkativecharacters['Character'], test_size=0.2)

In [14]:
cv = CountVectorizer(stop_words="english")
X_trained = cv.fit_transform(X_train)
X_tested = cv.transform(X_test)

In [15]:
dc = DummyClassifier()
dc.fit(X_trained, y_train)
pred = dc.predict(X_tested)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

    Butters       0.22      0.21      0.21       512
    Cartman       0.18      0.19      0.19       470
       Kyle       0.17      0.17      0.17       484
      Randy       0.18      0.19      0.19       455
       Stan       0.20      0.20      0.20       479

avg / total       0.19      0.19      0.19      2400



Дамми показал, что разброс не большой.

In [21]:
rfcl = RandomForestClassifier()
rfcl.fit(X_trained,  y_train)
pred = rfcl.predict(X_tested)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

    Butters       0.50      0.42      0.46       512
    Cartman       0.41      0.37      0.39       470
       Kyle       0.31      0.44      0.36       484
      Randy       0.46      0.38      0.42       455
       Stan       0.28      0.28      0.28       479

avg / total       0.39      0.38      0.38      2400



In [ ]:
Лес определяет достаточно плохо по сравнению с другими.

In [27]:
naive_model = MultinomialNB()
naive_model.fit(X_trained,  y_train)
pred = naive_model.predict(X_tested)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

    Butters       0.63      0.53      0.58       512
    Cartman       0.45      0.57      0.50       470
       Kyle       0.42      0.32      0.36       484
      Randy       0.41      0.55      0.47       455
       Stan       0.40      0.33      0.36       479

avg / total       0.46      0.46      0.46      2400



Наибольшая точность!

In [23]:
lr = LogisticRegression()
lr.fit(X_trained,  y_train)
pred = lr.predict(X_tested)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

    Butters       0.60      0.52      0.55       512
    Cartman       0.47      0.48      0.48       470
       Kyle       0.35      0.42      0.38       484
      Randy       0.49      0.45      0.47       455
       Stan       0.32      0.33      0.33       479

avg / total       0.45      0.44      0.44      2400



Относительно неплохой результат, но всё же уступает наивному Байесу :(

In [ ]:
Наиточнейшие результаты - в Байесе.